# Encoding the Molecular Hamiltonian 

This notebook shows how to use a Fermion-Qubit encoding to encode a second quantised Molecular hamiltonian.

$$H = \sum_{i,j} h_{ij}a^{\dagger}_i a_j + \sum_{i,j,k,l} h_{ijkl}a^{\dagger}_i a^{\dagger}_j a_k a_l $$

## Simple Useage

Let's first get the coefficients for our second quantised hamiltonian.

For the time being we can use randomly generated ones.

In [1]:
import numpy as np
constant_energy = 0
one_e_coeffs = np.random.random((3,3))
two_e_coeffs = np.random.random((3,3,3,3))


Rather than figure out how many modes we need for an encoding, we can create one of the right size by passing in our coefficients.

In [2]:
from ferrmion import TernaryTree
tt = TernaryTree.from_hamiltonian_coefficients((one_e_coeffs, two_e_coeffs))

To encode a hamiltonian, we need both a mapping from fermonic operators to qubits, and an enumeration scheme which accounts for the degrees of freedom when labeling modes and qubits.

For the mapping, we'll use JordanWigner and for the enumeration scheme, we can use the default one.

Note we can [optimise our encoded hamiltonian](https://ferrmion.readthedocs.io/en/latest/notebooks/pauli_weight.html) by getting more clever with these.

In [3]:
jw = tt.JW()
jw.enumeration_scheme = jw.default_enumeration_scheme()
jw.enumeration_scheme

{'': (0, 0), 'z': (1, 1), 'zz': (2, 2)}

In [4]:
from ferrmion.hamiltonians import molecular_hamiltonian
hamiltonian = molecular_hamiltonian(encoding=jw, one_e_coeffs=one_e_coeffs, two_e_coeffs=two_e_coeffs, constant_energy=constant_energy)
hamiltonian

{'YIX': -0.1585584023711703j,
 'ZZI': (-0.016437024527998084+0j),
 'ZXY': 0.008850858894941707j,
 'ZII': (-0.6881845456556445+0j),
 'IYX': 0.1403529732217562j,
 'IZZ': (0.1725175884719055+0j),
 'IXX': (0.303055426784816+0j),
 'YZX': 0.009343113568733685j,
 'XYZ': -0.02821483307621553j,
 'YXZ': 0.02821483307621553j,
 'ZYY': (-0.14067452177492276+0j),
 'XIY': 0.1585584023711703j,
 'IXY': -0.14035297322175622j,
 'IYY': (0.303055426784816+0j),
 'YIY': (-0.02879007367930804+0j),
 'XXZ': (0.22521815195640893+0j),
 'YXI': 0.18296801326390472j,
 'ZYX': -0.008850858894941707j,
 'ZIZ': (0.3598295496448026+0j),
 'XYI': -0.18296801326390472j,
 'XZX': (0.3007012564102717+0j),
 'IIZ': (-0.9869968379031477+0j),
 'XXI': (0.015505570946938968+0j),
 'YYI': (0.015505570946938912+0j),
 'ZXX': (-0.14067452177492276+0j),
 'XZY': -0.009343113568733658j,
 'XIX': (-0.028790073679308054+0j),
 'III': (1.4650206579283944+0j),
 'YYZ': (0.22521815195640904+0j),
 'IZI': (-0.3057493879583123+0j),
 'YZY': (0.300701256

## Hamiltonian Templates

Sometimes it can be useful to find out how an encoding behaves without providing coefficients. This lets us see which terms of the second quantised hamiltonian contribute to which pauli terms of the qubit Hamiltonian.

### Creating a Template

In [5]:
from ferrmion.hamiltonians import molecular_hamiltonian_template

The only information we need to provide is a set of XZ-encoded vectors (this is how ferrmion manipulates encodings internally) and imaginary factors for these vectors. 

In [6]:
ipowers, symplectics = jw._build_symplectic_matrix()

In [7]:
ipowers

array([0, 1, 0, 1, 0, 1], dtype=uint8)

In [8]:
np.array(symplectics, dtype=int)

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 1, 0],
       [0, 0, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1]])

In [9]:
template = molecular_hamiltonian_template(ipowers, symplectics, physicist_notation=True)
template

{'ZII': {(1, 0, 1, 0): (0.25+0j),
  (0, 0, 0, 0): 0j,
  (2, 2, 0, 0): 0j,
  (1, 0, 0, 1): (-0.25+0j),
  (0, 1, 0, 1): (0.25+0j),
  (1, 1, 0, 0): 0j,
  (0, 1, 1, 0): (-0.25+0j),
  (0, 2, 2, 0): (-0.25+0j),
  (0, 0, 1, 1): 0j,
  (0, 2, 0, 2): (0.25+0j),
  (0, 0): (-0.5+0j),
  (0, 0, 2, 2): 0j,
  (2, 0, 0, 2): (-0.25+0j),
  (2, 0, 2, 0): (0.25+0j)},
 'YZX': {(0, 2, 1, 1): 0j,
  (1, 0, 1, 2): 0.125j,
  (1, 2, 1, 0): -0.125j,
  (2, 0, 2, 2): 0j,
  (0, 2): -0.25j,
  (1, 1, 0, 2): 0j,
  (2, 0, 1, 1): 0j,
  (2, 1, 0, 1): -0.125j,
  (0, 1, 2, 1): 0.125j,
  (0, 2, 2, 2): 0j,
  (1, 0, 2, 1): -0.125j,
  (0, 0, 0, 2): 0j,
  (0, 1, 1, 2): -0.125j,
  (2, 0): 0.25j,
  (2, 2, 2, 0): 0j,
  (2, 2, 0, 2): 0j,
  (0, 0, 2, 0): 0j,
  (1, 2, 0, 1): 0.125j,
  (2, 1, 1, 0): 0.125j,
  (1, 1, 2, 0): 0j,
  (0, 2, 0, 0): 0j,
  (2, 0, 0, 0): 0j},
 'ZYX': {(0, 0, 2, 1): 0j,
  (0, 1, 0, 2): -0.125j,
  (1, 0, 0, 2): 0.125j,
  (0, 2, 1, 0): -0.125j,
  (2, 0, 0, 1): -0.125j,
  (0, 2, 0, 1): 0.125j,
  (1, 0, 2, 0): -0.125

### Filling a Template

So we can now see which modes contribute to each Pauli operator.

We can now fill out our template with the coefficients we used earlier.

This time we only need to provide a mapping from fermionic modes to majorana operators. Again we can use the default.

In [10]:
jw.default_mode_op_map

{0: 0, 1: 1, 2: 2}

In [11]:
from ferrmion.hamiltonians import fill_template
filled_template = fill_template(template,constant_energy=constant_energy, one_e_coeffs=one_e_coeffs, two_e_coeffs=two_e_coeffs, mode_op_map=jw.default_mode_op_map)
filled_template

{'YXI': 0.18296801326390472j,
 'XIX': (-0.028790073679308067+0j),
 'XXI': (0.015505570946938968+0j),
 'ZZI': (-0.016437024527998084+0j),
 'YYI': (0.015505570946938926+0j),
 'ZII': (-0.6881845456556444+0j),
 'XYZ': -0.02821483307621553j,
 'IXX': (0.303055426784816+0j),
 'ZYX': -0.008850858894941693j,
 'XYI': -0.18296801326390472j,
 'IZI': (-0.3057493879583123+0j),
 'ZIZ': (0.3598295496448026+0j),
 'YIX': -0.1585584023711703j,
 'IIZ': (-0.986996837903148+0j),
 'XXZ': (0.225218151956409+0j),
 'IXY': -0.14035297322175622j,
 'XZX': (0.3007012564102718+0j),
 'XZY': -0.0093431135687337j,
 'YZX': 0.009343113568733685j,
 'YYZ': (0.225218151956409+0j),
 'ZXY': 0.008850858894941721j,
 'IYX': 0.1403529732217562j,
 'ZYY': (-0.14067452177492273+0j),
 'IYY': (0.303055426784816+0j),
 'YXZ': 0.02821483307621557j,
 'YIY': (-0.028790073679308054+0j),
 'YZY': (0.3007012564102718+0j),
 'ZXX': (-0.14067452177492276+0j),
 'III': (1.4650206579283944+0j),
 'IZZ': (0.1725175884719055+0j),
 'XIY': 0.158558402371

Let' check to see that we have the same hamiltonian in each.

In [12]:
assert hamiltonian.keys() == filled_template.keys()
for k in hamiltonian.keys():
    # There is a little numerical instability so 
    # some values are different by 1e-18 or so!
    assert np.isclose(hamiltonian[k], filled_template[k])